## Setup
This notebook requires that you have a copy of the AACT data set up in a MySQL database. To make this as easy as possible I've created some simple scripts to perform all necessary tasks to download, extract and transform the data into a MySql database. The code and directions can be found at: [https://github.com/dansbits/aact_analysis](https://github.com/dansbits/aact_analysis)

This analysis was completed using the March 2016 release of AACT.

In [1]:
require 'dbi'
require 'daru'
require 'nyaplot'
require 'yaml'

# change this to point to your aact_analysis directory
database = YAML.load_file("../secrets.yml")['database']; nil

db = DBI.connect("dbi:Mysql:#{database['name']}:#{database['host']}", database['user'], database['password']); nil

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

## ClinicalTrials.gov - FDAAA Section 801 Compliance

This analysis takes a look at the Food, Drug and Cosmetic Adminisration Act and compliance with some of the regulations it puts in place. Specifically I'm interested in measuring adherence with posting of results to the results data bank.

It first became possible to post results to ClinicalTrials.gov in September 2008. I'd first like to look into how often trials post results without taking into account whether it's required or not.

In [2]:
studies = Daru::DataFrame.from_sql(
  db, 
  <<-SQL
    select
      cs.nct_id,
      start_date,
      primary_completion_date,
      firstreceived_results_date,
      is_section_801,
      case 
        when firstreceived_results_date is null then 'No results submitted'
        when firstreceived_results_date is not null then 'Results submitted' 
      end as results_status,
      case
        when firstreceived_results_date is not null and firstreceived_results_date <= date_add(primary_completion_date, INTERVAL 2 YEAR) then 'Results submitted'
        else 'No results submitted'
      end as has_2yr_results
    from clinical_study cs
    where
      -- include studies which started after 9/1/2008 or completed after completed after 9/1/2008
      (
        start_date is not null 
        and start_date > '2008-09-01'
      )
      and (
        primary_completion_date is not null 
        and primary_completion_date < '2014-01-01'
      )
  SQL
)

studies.size

48310

In [3]:
overall_results_rates = studies.group_by(:has_2yr_results).count[1..1]
overall_results_rates.vectors = Daru::Index.new([:total_study_count])
overall_results_rates[:results_status] = overall_results_rates.index.to_a

overall_results_rates[:percent_of_total] = overall_results_rates.map(:row) do |r| 
  ((r[:total_study_count].to_f / studies.size.to_f) * 100.0).round(4)
end

overall_results_rates.plot type: :bar, x: :results_status, y: :total_study_count do |plot, diagram|
  plot.x_label "Results Status"
  plot.y_label "Total Studies"
  diagram.color ['#77DD77']
end

#<Nyaplot::Frame:0x0000000571a5e8 @properties={:panes=>[#<Nyaplot::Plot:0x00000005732968 @properties={:diagrams=>[#<Nyaplot::Diagram:0x00000005721280 @properties={:type=>:bar, :options=>{:x=>:results_status, :y=>:total_study_count, :color=>["#77DD77"]}, :data=>"abed8eb2-66e5-4124-b913-656816dc5a7f"}, @xrange=["No results submitted", "Results submitted"], @yrange=[0, 42448]>], :options=>{:x_label=>"Results Status", :y_label=>"Total Studies", :width=>700, :xrange=>["No results submitted", "Results submitted"], :yrange=>[0, 42448]}}>], :data=>{"abed8eb2-66e5-4124-b913-656816dc5a7f"=>#<Nyaplot::DataFrame:0x00000005722bf8 @name="abed8eb2-66e5-4124-b913-656816dc5a7f", @rows=[{:total_study_count=>42448, :results_status=>"No results submitted", :percent_of_total=>87.8659}, {:total_study_count=>5862, :results_status=>"Results submitted", :percent_of_total=>12.1341}]>}, :extension=>[]}>

In [5]:
overall_results_rates

#<Daru::DataFrame(2x3)>
            total_stud results_st percent_of
 No results      42448 No results    87.8659
 Results su       5862 Results su    12.1341

In [6]:
studies[:completion_year] = studies.map(:row) { |r| r[:primary_completion_date].year }

results_rates_by_year = studies.group_by(:completion_year, :has_2yr_results).count[1..1]
results_rates_by_year.vectors = Daru::Index.new([:total_study_count])
results_rates_by_year[:completion_year] = results_rates_by_year.index.to_a.map { |i| i[0] }
results_rates_by_year[:results_status] = results_rates_by_year.index.to_a.map { |i| i[1] }

results_rates_by_year[:yearly_percent] = results_rates_by_year.map(:row) do |year_row|
  ((year_row[:total_study_count].to_f / studies.filter(:row) { |study_row| year_row[:completion_year] == study_row[:completion_year] }.size.to_f) * 100.0).round(4)
end
    
plot_data = results_rates_by_year.filter(:row) { |r| r[:results_status] == 'Results submitted' }
plot_data[:completion_year] = plot_data[:completion_year].map { |y| y.to_s }

plot_data.plot type: :bar, x: :completion_year, y: :yearly_percent do |plot, diagram|
  plot.x_label "Primary Completion Year"
  plot.y_label "% of Studies with Posted Results"
#   plot.title "Annual Rates of Posting Results within 2 Years of Study Comopletion"
  diagram.color ['#77DD77']
end

[:diagrams, :filter, :options, :width, :height, :margin, :xrange, :yrange, :x_label, :y_label, :bg_color, :grid_color, :legend, :legend_width, :legend_options, :zoom, :rotate_x_label, :rotate_y_label, :x_scale, :y_scale, :add, :add_with_df, :to_iruby, :show, :export_html, :df_list, :before_to_json, :configure, :init_properties, :to_json, :set_property, :get_property, :dclone, :itself, :try, :psych_to_yaml, :to_yaml, :to_yaml_properties, :pretty_print, :pretty_print_cycle, :pretty_print_instance_variables, :pretty_print_inspect, :nil?, :===, :=~, :!~, :eql?, :hash, :<=>, :class, :singleton_class, :clone, :dup, :taint, :tainted?, :untaint, :untrust, :untrusted?, :trust, :freeze, :frozen?, :to_s, :inspect, :methods, :singleton_methods, :protected_methods, :private_methods, :public_methods, :instance_variables, :instance_variable_get, :instance_variable_set, :instance_variable_defined?, :remove_instance_variable, :instance_of?, :kind_of?, :is_a?, :tap, :send, :public_send, :respond_to?, :e

#<Nyaplot::Frame:0x00000004c9dd40 @properties={:panes=>[#<Nyaplot::Plot:0x00000004b103b0 @properties={:diagrams=>[#<Nyaplot::Diagram:0x00000004c0c368 @properties={:type=>:bar, :options=>{:x=>:completion_year, :y=>:yearly_percent, :color=>["#77DD77"]}, :data=>"886c3f8b-e8ea-41e8-8f53-620083d134fa"}, @xrange=["2008", "2009", "2010", "2011", "2012", "2013"], @yrange=[0, 12.8311]>], :options=>{:x_label=>"Primary Completion Year", :y_label=>"% of Studies with Posted Results", :width=>700, :xrange=>["2008", "2009", "2010", "2011", "2012", "2013"], :yrange=>[0, 12.8311]}}>], :data=>{"886c3f8b-e8ea-41e8-8f53-620083d134fa"=>#<Nyaplot::DataFrame:0x00000004bfd340 @name="886c3f8b-e8ea-41e8-8f53-620083d134fa", @rows=[{:total_study_count=>13, :completion_year=>"2008", :results_status=>"Results submitted", :yearly_percent=>10.0775}, {:total_study_count=>307, :completion_year=>"2009", :results_status=>"Results submitted", :yearly_percent=>9.7833}, {:total_study_count=>873, :completion_year=>"2010", :results_status=>"Results submitted", :yearly_percent=>12.125}, {:total_study_count=>1187, :completion_year=>"2011", :results_status=>"Results submitted", :yearly_percent=>11.4288}, {:total_study_count=>1593, :completion_year=>"2012", :results_status=>"Results submitted", :yearly_percent=>12.5088}, {:total_study_count=>1889, :completion_year=>"2013", :results_status=>"Results submitted", :yearly_percent=>12.8311}]>}, :extension=>[]}>

This shows that a smaller percentage of studies have been posting results since 2008. It's important to note though that posting results is not required for all of these studies and those plot doesn't consider what percentage of studies are required to share results. While many would argue there is an ethical obligation for all studies to post results, there is not a legal mandate for all studies.

Its also worth noting that there have been increasingly
more studies registered and completed on ClinicalTrials.gov each year since 2008.

In [7]:
yearly_study_completions = studies.group_by(:completion_year).count[1..1]
yearly_study_completions.vectors = Daru::Index.new([:total_study_count])
yearly_study_completions[:completion_year] = yearly_study_completions.index.to_a.map { |y| y.to_s }
yearly_study_completions

yearly_study_completions.plot type: :bar, x: :completion_year, y: :total_study_count do |plot, diagram|
  plot.x_label 'Year'
  plot.y_label 'Total studies completed'
  diagram.color ['#77DD77']
end

#<Nyaplot::Frame:0x00000007584ad8 @properties={:panes=>[#<Nyaplot::Plot:0x0000000757c7c0 @properties={:diagrams=>[#<Nyaplot::Diagram:0x00000007585190 @properties={:type=>:bar, :options=>{:x=>:completion_year, :y=>:total_study_count, :color=>["#77DD77"]}, :data=>"42edd36c-2246-44e2-b708-f8847e68d3b2"}, @xrange=["2008", "2009", "2010", "2011", "2012", "2013"], @yrange=[0, 14722]>], :options=>{:x_label=>"Year", :y_label=>"Total studies completed", :width=>700, :xrange=>["2008", "2009", "2010", "2011", "2012", "2013"], :yrange=>[0, 14722]}}>], :data=>{"42edd36c-2246-44e2-b708-f8847e68d3b2"=>#<Nyaplot::DataFrame:0x00000007585848 @name="42edd36c-2246-44e2-b708-f8847e68d3b2", @rows=[{:total_study_count=>129, :completion_year=>"2008"}, {:total_study_count=>3138, :completion_year=>"2009"}, {:total_study_count=>7200, :completion_year=>"2010"}, {:total_study_count=>10386, :completion_year=>"2011"}, {:total_study_count=>12735, :completion_year=>"2012"}, {:total_study_count=>14722, :completion_year=>"2013"}]>}, :extension=>[]}>

# FDAAA Section 801 Compliance

The above analysis shows that results are submitted by very few studies overall. However, it's not required for all studies. For many studies included above there is no legal obligation to post results. Let's take a look at those which ARE required to submit results and see how compliant they are. In September 2009 it became a requirement that applicable studies under FDAAA section 801 submit adverse events and final results to the registry.

Studies are not required to submit results until 1 year after the primary completion date so I will exclude studies completed studies which have not passed this grace period yet.

In [5]:
regulated_studies = Daru::DataFrame.from_sql(
  db, 
  <<-SQL
    select
  cs.nct_id,
  start_date,
  primary_completion_date,
  firstreceived_results_date,
  overall_status,
  DATE_ADD(primary_completion_date, INTERVAL 1 YEAR) AS results_due_date,
  is_section_801,
  enrollment,
  enrollment_type,
  case 
    when firstreceived_results_date is null or firstreceived_results_date = '' then 'No results submitted'
    when firstreceived_results_date is not null and firstreceived_results_date <= DATE_ADD(primary_completion_date, INTERVAL 1 YEAR)  then 'Results submitted on time' 
    when firstreceived_results_date is not null and firstreceived_results_date > DATE_ADD(primary_completion_date, INTERVAL 1 YEAR)  then 'Results submitted after due' 
  end as results_status
from clinical_study cs
where
  -- include only trials that are required to submit results and who completed over a year ago. Trials are
  -- allowed 1 year from the primary completion date to submit results:
  -- 1. fits FDAAA 801 definition of an "applicable clinical trial"
  -- 2. initiated after 9/27/2007 
  --    or completed after 12/26/2017 even if initiated before 9/27/2007
  is_section_801 = 'Yes'
  AND (
    (
      start_date is not null 
      and start_date > '2007-09-27'
    )
    and (
      primary_completion_date is not null 
      and primary_completion_date > '2007-12-26'
      and primary_completion_date < '2015-01-01'
    )
    
  )
  -- investigational drugs which are seeking FDA approval are permitted to withold results until
  -- an approval decision has been made. According to the ClinicalTrials.gov xml schema, these studies
  -- will have a title of [Trial of device that is not approved or cleared by the U.S. FDA]. I'm excluding
  -- these studies from analysis because they are not required to post results at this time. 
  AND brief_title != '[Trial of device that is not approved or cleared by the U.S. FDA]'
  SQL
)

# 13,487 in total
regulated_studies.size

16788

In [10]:
overall_results_rates = regulated_studies.group_by(:results_status).count[1..1]
overall_results_rates.vectors = Daru::Index.new([:total_study_count])
overall_results_rates[:results_status] = overall_results_rates.index.to_a

overall_results_rates[:percent_of_total] = overall_results_rates.map(:row) do |r| 
  ((r[:total_study_count].to_f / regulated_studies.size.to_f) * 100.0).round(4)
end

overall_results_rates.plot type: :bar, x: :results_status, y: :total_study_count do |plot, diagram|
  plot.x_label "Results Status"
  plot.y_label "Total Studies"
  diagram.color ['#77DD77']
end

#<Nyaplot::Frame:0x0000000a4beb28 @properties={:panes=>[#<Nyaplot::Plot:0x0000000a4b8d90 @properties={:diagrams=>[#<Nyaplot::Diagram:0x0000000a4bf078 @properties={:type=>:bar, :options=>{:x=>:results_status, :y=>:total_study_count, :color=>["#77DD77"]}, :data=>"a6fb4fc6-dc92-410a-a117-217748c14826"}, @xrange=["No results submitted", "Results submitted after due", "Results submitted on time"], @yrange=[0, 9933]>], :options=>{:x_label=>"Results Status", :y_label=>"Total Studies", :width=>700, :xrange=>["No results submitted", "Results submitted after due", "Results submitted on time"], :yrange=>[0, 9933]}}>], :data=>{"a6fb4fc6-dc92-410a-a117-217748c14826"=>#<Nyaplot::DataFrame:0x0000000a4bf488 @name="a6fb4fc6-dc92-410a-a117-217748c14826", @rows=[{:total_study_count=>9933, :results_status=>"No results submitted", :percent_of_total=>59.1673}, {:total_study_count=>5382, :results_status=>"Results submitted after due", :percent_of_total=>32.0586}, {:total_study_count=>1473, :results_status=>"Results submitted on time", :percent_of_total=>8.7741}]>}, :extension=>[]}>

In [11]:
overall_results_rates

#<Daru::DataFrame(3x3)>
            total_stud results_st percent_of
 No results       9933 No results    59.1673
 Results su       5382 Results su    32.0586
 Results su       1473 Results su     8.7741

11,084 (61.4%) of these studies have no results posted. 4,401 (24.39%) have submitted results but not until after the due date. Finally 2561 (14.2%) posted results within the 1 year period following completion of the study.

This seems low but one further consideration to be taken is that some studies are terminated or suspended before ever enrolling any participants so perhaps some truly have no data to report. What happens if we limit the above plot to only include studies with a status of 'Completed' - meaning that it the study concluded normally.

In [15]:
completed_regulated_studies = regulated_studies.filter(:row) { |r| r[:overall_status] == 'Completed' }
p completed_regulated_studies.size
overall_results_rates = completed_regulated_studies.group_by(:results_status).count[1..1]
overall_results_rates.vectors = Daru::Index.new([:total_study_count])
overall_results_rates[:results_status] = overall_results_rates.index.to_a

overall_results_rates[:percent_of_total] = overall_results_rates.map(:row) do |r| 
  ((r[:total_study_count].to_f / completed_regulated_studies.size.to_f) * 100.0).round(4)
end

overall_results_rates.plot type: :bar, x: :results_status, y: :total_study_count do |plot, diagram|
  plot.x_label "Results Status"
  plot.y_label "Total Studies"
  diagram.color ['#77DD77']
end

11643


#<Nyaplot::Frame:0x00000007e22d48 @properties={:panes=>[#<Nyaplot::Plot:0x00000007e257c8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x00000007e23338 @properties={:type=>:bar, :options=>{:x=>:results_status, :y=>:total_study_count, :color=>["#77DD77"]}, :data=>"1e6a2550-aaf6-475a-be09-fe0b54f5e14d"}, @xrange=["No results submitted", "Results submitted after due", "Results submitted on time"], @yrange=[0, 6110]>], :options=>{:x_label=>"Results Status", :y_label=>"Total Studies", :width=>700, :xrange=>["No results submitted", "Results submitted after due", "Results submitted on time"], :yrange=>[0, 6110]}}>], :data=>{"1e6a2550-aaf6-475a-be09-fe0b54f5e14d"=>#<Nyaplot::DataFrame:0x00000007e237e8 @name="1e6a2550-aaf6-475a-be09-fe0b54f5e14d", @rows=[{:total_study_count=>6110, :results_status=>"No results submitted", :percent_of_total=>52.4779}, {:total_study_count=>4369, :results_status=>"Results submitted after due", :percent_of_total=>37.5247}, {:total_study_count=>1164, :results_status=>"Results submitted on time", :percent_of_total=>9.9974}]>}, :extension=>[]}>

In [14]:
overall_results_rates

#<Daru::DataFrame(3x3)>
            total_stud results_st percent_of
 No results       6110 No results    52.4779
 Results su       4369 Results su    37.5247
 Results su       1164 Results su     9.9974